In [0]:
# %sql
# CREATE TABLE aarthi.default.sku_table (
#     sku_id INT PRIMARY KEY,
#     product_name VARCHAR(255) NOT NULL,
#     sku_code VARCHAR(100) NOT NULL,
#     quantity INT NOT NULL,
#     price DECIMAL(10, 2) NOT NULL
# );

In [0]:
# %sql
# INSERT INTO aarthi.default.sku_table (sku_id, product_name, sku_code, quantity, price)
# VALUES
#     (1,'Laptop', 'SKU001', 10, 800.00),
#     (2,'Smartphone', 'SKU002', 50, 500.00),
#     (3,'Headphones', 'SKU003', 30, 100.00),
#     (4,'Monitor', 'SKU004', 15, 300.00);

In [0]:
# %sql
# CREATE TABLE aarthi.default.revenue_table (
#     revenue_id INT PRIMARY KEY,
#     sku_id INT,
#     sale_amount INT NOT NULL,
#     revenue DECIMAL(10, 2) NOT NULL,
#     FOREIGN KEY (sku_id) REFERENCES aarthi.default.sku_table(sku_id)
# );

In [0]:
# %sql
# INSERT INTO aarthi.default.revenue_table (revenue_id, sku_id, sale_amount, revenue)
# VALUES
#     (11, 1, 10, 8000.00),  -- 10 units of Laptop, total revenue = 10 * 800.00
#     (12, 2, 30, 15000.00), -- 30 units of Smartphone, total revenue = 30 * 500.00
#     (13, 3, 20, 2000.00),  -- 20 units of Headphones, total revenue = 20 * 100.00
#     (14, 4, 10, 3000.00);  -- 10 units of Monitor, total revenue = 10 * 300.00

In [0]:
# %sql
# SELECT 
#     SUM(r.revenue) AS total_revenue,
#     SUM(s.quantity * s.price) AS calculated_revenue
# FROM aarthi.default.revenue_table r
# JOIN aarthi.default.sku_table s ON r.sku_id = s.sku_id;

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("Create Revenue Tables").getOrCreate()

# Sample data for the `sku` table (product details)
sku_data = [
    ('sku_001', 'Product A', 10, 5.0),  # (sku_id, product_name, quantity, price)
    ('sku_002', 'Product B', 20, 3.0),
    ('sku_003', 'Product C', 15, 7.0)
]

# Define the schema for the `sku` table
sku_columns = ['sku_id', 'product_name', 'quantity', 'price']

# Create DataFrame for the `sku` table
sku_df = spark.createDataFrame(sku_data, schema=sku_columns)

# Show the `sku` table
sku_df.show()

# Sample data for the `revenue` table (revenue details for products)
revenue_data = [ 
    ('sku_001', 50.0),  # (sku_id, revenue)
    ('sku_002', 60.0),
    ('sku_003', 106.0)
]

# Define the schema for the `revenue` table
revenue_columns = ['sku_id', 'revenue']

# Create DataFrame for the `revenue` table
revenue_df = spark.createDataFrame(revenue_data, schema=revenue_columns)

# Show the `revenue` table
revenue_df.show()

# Perform Regression Test: Compare the total revenue from the `revenue` table and calculated revenue from `sku` table

# Calculate the total revenue from the `revenue` table
total_revenue_from_revenue_table = revenue_df.agg({'revenue': 'sum'}).collect()[0][0]

# Calculate the revenue based on `sku` table (quantity * price)
calculated_revenue = sku_df.withColumn('calculated_revenue', col('quantity') * col('price')) \
                           .agg({'calculated_revenue': 'sum'}).collect()[0][0]

print(f"Total revenue from revenue table: {total_revenue_from_revenue_table}")
print(f"Calculated revenue based on SKU table: {calculated_revenue}")

# Perform regression test
if total_revenue_from_revenue_table == calculated_revenue:
    print("Regression test passed: Revenue is consistent")
else:
    print(f"Regression test failed: {total_revenue_from_revenue_table} != {calculated_revenue}")
    raise Exception("Regression test failed")
